In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

Using TensorFlow backend


C:\Users\Ethan\AppData\Local\Temp\ipykernel_10124\3283149789.py:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
cle = pd.read_csv('cle_metadata_dnn.csv')
vir = pd.read_csv('vir_metadata_dnn.csv')
hun = pd.read_csv('hun_metadata_dnn.csv')
swi = pd.read_csv('swi_metadata_dnn.csv')

In [3]:
cle_train,cle_test = train_test_split(cle,test_size=0.33, random_state=42)
vir_train,vir_test = train_test_split(vir,test_size=0.33, random_state=42)
hun_train,hun_test = train_test_split(hun,test_size=0.33, random_state=42)
swi_train,swi_test = train_test_split(swi,test_size=0.33, random_state=42)

In [4]:
Train = pd.concat([cle_train,vir_train,hun_train])
Test = pd.concat([cle_test,vir_test,hun_test,swi_test,swi_train])

In [5]:
X_train = Train.iloc[:,:-1]
X_test = Test.iloc[:,:-1]

y_train = Train.iloc[:,-1]
y_test = Test.iloc[:,-1]

Y_train_binary = y_train.apply(lambda x: 1 if x > 0 else 0)
Y_test_binary = y_test.apply(lambda x: 1 if x > 0 else 0)

# CNN

In [26]:
# Define your CNN model function
def build_model(hp):
    model = keras.Sequential()
    
    # Define the hyperparameter search space
    hp_filters = hp.Int('num_filters', min_value=32, max_value=128, step=32)
    hp_kernel_size = hp.Int('kernel_size', min_value=3, max_value=5)
    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    hp_dropout = hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)
    
    # Add convolutional layers
    model.add(layers.Conv1D(hp_filters, hp_kernel_size, activation='relu', input_shape=(64,1)))
    model.add(layers.Conv1D(hp_filters, hp_kernel_size, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
              
    model.add(layers.Conv1D(hp_filters, hp_kernel_size, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    
    # Add fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(hp_units, activation='relu'))
    model.add(layers.Dense(2, activation='sigmoid'))
    
    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# Create a RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',  # Hyperparameter optimization goal
    max_trials=100,             # Number of trials (random combinations of hyperparameters)
    directory='random_search', # Directory to save results
    project_name='cnn_tuning'  # Name of the tuning project
)


# Perform the hyperparameter search
tuner.search(X_train, Y_train_binary, epochs=20, validation_split=0.2)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

#display the best model
best_model.summary()

Trial 100 Complete [00h 00m 03s]
val_accuracy: 0.822429895401001

Best val_accuracy So Far: 0.9065420627593994
Total elapsed time: 00h 05m 04s
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 60, 96)            576       
                                                                 
 conv1d_4 (Conv1D)           (None, 56, 96)            46176     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 28, 96)           0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 24, 96)            46176     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 12, 96)           0         
 1D)                                       

In [27]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

best_model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = best_model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred, digits=4))

Epoch 1/1000
17/17 [==============================] - 1s 8ms/step - loss: 0.6961 - accuracy: 0.4878
Epoch 2/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.6929 - accuracy: 0.5197
Epoch 3/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.6925 - accuracy: 0.5141
Epoch 4/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.6909 - accuracy: 0.5178
Epoch 5/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.6899 - accuracy: 0.5347
Epoch 6/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.6840 - accuracy: 0.6116
Epoch 7/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.6700 - accuracy: 0.6285
Epoch 8/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.6335 - accuracy: 0.6529
Epoch 9/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.5875 - accuracy: 0.6961
Epoch 10/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.5591 - accuracy: 0.7542

17/17 [==============================] - 0s 8ms/step - loss: 0.1522 - accuracy: 0.9550
Epoch 83/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.1548 - accuracy: 0.9493
Epoch 84/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.1639 - accuracy: 0.9531
Epoch 85/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.1806 - accuracy: 0.9381
Epoch 86/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.1508 - accuracy: 0.9568
Epoch 87/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.1581 - accuracy: 0.9400
Epoch 88/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.1494 - accuracy: 0.9456
Epoch 89/1000
17/17 [==============================] - 0s 8ms/step - loss: 0.1452 - accuracy: 0.9587
Epoch 90/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.1538 - accuracy: 0.9437
Epoch 91/1000
17/17 [==============================] - 0s 7ms/step - loss: 0.1283 - accuracy: 0.9662
Epoc